# Purpose
This notebook will allow for the creation of sample documents that can then be used in the other notebooks. The purpose here is to experiment with using GenAI to also create sample data for demonstrations.

## Setup
The functions defined below will use Chat Completions to create a series of status reports about a software project based on the initial prompt. A loop will keep adding the previous report to ensure consistency. The reports are then written to the local file system as both the original Markdown and also PDF.

In [ ]:
!pip install pypdf reportlab markdown python-dotenv openai --upgrade

## Ensure imports are present and correct

In [ ]:
from openai import AzureOpenAI
import os
from pypdf import PdfWriter
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph
import markdown
from dotenv import load_dotenv

## Environment Setup
This notebook leverages .env files and uses relative pathing to select the correct one. It is intended that the .env file exist in the same level of the folder structure as the notebook itself

In [ ]:
# Load environment variables from .env file
load_dotenv(dotenv_path=".env")

# Function to generate a prompt and get a response from Azure OpenAI
def generate_prompt(prompt):
    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
        api_version="2024-02-01"
    )
    
    conversation=[
            {"role": "system", "content": 
             """You are a helpful assistant. You will help create synthetic data. 
             The data you will create are status reports for a project to improve efficiency in a company call center. The software development project is looking to add AI capabilities to an existing applications. The
             team doing the work are experienced developers but new to generative AI solutions. To compensate a team of data scientists and data engineers have been integrated
             into the larger team. The goal of the project is reduce time to resolution for a support case thus increasing customer satisfaction and improving the work life of the call center agents. The project is in its final weeks and 
             numerous issues have been identified. **BE CREATIVE WITH THE ISSUES YOU CREATE**. The project is in a critical phase and leadership wants daily updates. The status report will include the following sections: 1. Project Overview, 2. Current Status, 3. Issues and Risks, 4. Next Steps. IMPORTANT: The response should be in Markdown format and contain nothing more than what the report specifies."""},
            {"role": "user", "content": prompt}]

    combined_report = ""
    for i in range(4):
        response = client.chat.completions.create(
            model=os.getenv("AZURE_OPENAI_MODEL"),
            messages=conversation
        )

        write_to_markdown(response.choices[0].message.content, f"call-center-status{i}.md")
        combined_report += response.choices[0].message.content
        write_to_pdf(response.choices[0].message.content, f"call-center-status-report{i}.pdf")

        conversation.append({"role": "assistant", "content": response.choices[0].message.content})
        conversation.append({"role": "user", "content": "Based on the previous report, create a new report with the same sections but different content that reflects the current status of the project."})

    # Write the final report to a PDF file
    write_to_pdf(combined_report, "call-center-status-report.pdf")

# Function to write the output to a PDF file
def write_to_pdf(text, filename):
    # Convert markdown to HTML
    html_text = markdown.markdown(text)
    
    # Create a PDF document
    pdf = SimpleDocTemplate(os.path.join('..', 'examples', filename), pagesize=letter)
    styles = getSampleStyleSheet()
    story = [Paragraph(text, styles["Normal"])]
    
    # Build the PDF
    pdf.build(story)

# Function to write the output to a Markdown file
def write_to_markdown(text, filename):
    with open(os.path.join('..', 'examples', filename), 'w') as f:
        f.write(markdown.markdown(text))


# Main function to run the script
if __name__ == "__main__":
    prompt = "Create a status report for a software development project. Assume the project is in a critical phase and leadership wants daily updates. The current status reflects ongoing challenges with defect burndown and challenges with the new feature."
    generate_prompt(prompt)